# Assignment 2

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./05_src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model]() or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Place the files in the ../../05_src/data/fires/ directory and load the appropriate file. 

In [1]:
# Load the libraries as required.
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler, OneHotEncoder, PowerTransformer
from sklearn.linear_model import Ridge, TheilSenRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, log_loss, cohen_kappa_score, f1_score
from scipy import stats
import numpy as np
from sklearn.compose import ColumnTransformer
import pickle

In [ ]:
# Load data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = 0, names = columns)) 
fires_dt.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi      517 non-null    float64
 8   temp     517 non-null    float64
 9   rh       517 non-null    int64  
 10  wind     517 non-null    float64
 11  rain     517 non-null    float64
 12  area     517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


# Get X and Y

Create the features data frame and target data.

In [ ]:
X = fires_dt.drop(columns = ['area']) # area is the target variable, as per the question
y = fires_dt['area']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
# we are splitting the data before preprocessing to avoid data leakage

In [ ]:
X_num = (X.select_dtypes(include="number"))
Q1 = X_num.quantile(0.25)
Q3 = X_num.quantile(0.75)
IQR = Q3 - Q1
((X_num < (Q1 - 1.5 * IQR)) | (X_num > (Q3 + 1.5 * IQR))).sum() # check for outliers outside 1.5*IQR range.
# it seems there's a lot of outliers in the numerical columns. We will use RobustScaler to scale the numerical columns

coord_x     0
coord_y    51
ffmc       53
dmc        17
dc          8
isi        23
temp        2
rh         12
wind       13
rain        8
dtype: int64

# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [ ]:
preproc1 = ColumnTransformer(
    transformers=[
        ('num', RobustScaler(), X.select_dtypes(include="number").columns.values), # RobustScaler for numerical columns
        ('cat', OneHotEncoder(handle_unknown='infrequent_if_exist', drop='if_binary'), # OneHotEncoder for categorical columns
         X.select_dtypes(include="object").columns.values)
    ], remainder='drop'
)


### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [ ]:
yj_pipeline = Pipeline(steps=[
    ('power', PowerTransformer(method='yeo-johnson')), # Yeo-Johnson transformation
    ('scaler', RobustScaler())
])

preproc2 = ColumnTransformer(
    transformers=[
        ('num', yj_pipeline, X.select_dtypes(include="number").columns.values),
        ('cat', OneHotEncoder(handle_unknown='infrequent_if_exist', drop='if_binary'), 
         X.select_dtypes(include="object").columns.values)
    ], remainder='drop'
)

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [ ]:
# Pipeline A = preproc1 + baseline
pipe_A = Pipeline(steps=[
    ('preproc', preproc1),
    ('regression', Ridge()) # using Ridge regression as baseline model
])

In [9]:
# Pipeline B = preproc2 + baseline
pipe_B = Pipeline(steps=[
    ('preproc', preproc2),
    ('regression', Ridge())
])

In [ ]:
# Pipeline C = preproc1 + advanced model
pipe_C = Pipeline(steps=[
    ('preproc', preproc1),
    ('regression', TheilSenRegressor()) # TheilSenRegressor as advanced model, due to its robustness to outliers
])

In [11]:
# Pipeline D = preproc2 + advanced model

pipe_D = Pipeline(steps=[
    ('preproc', preproc2),
    ('regression', TheilSenRegressor())
])

In [12]:
pipe_A.get_params().keys()

dict_keys(['memory', 'steps', 'transform_input', 'verbose', 'preproc', 'regression', 'preproc__force_int_remainder_cols', 'preproc__n_jobs', 'preproc__remainder', 'preproc__sparse_threshold', 'preproc__transformer_weights', 'preproc__transformers', 'preproc__verbose', 'preproc__verbose_feature_names_out', 'preproc__num', 'preproc__cat', 'preproc__num__copy', 'preproc__num__quantile_range', 'preproc__num__unit_variance', 'preproc__num__with_centering', 'preproc__num__with_scaling', 'preproc__cat__categories', 'preproc__cat__drop', 'preproc__cat__dtype', 'preproc__cat__feature_name_combiner', 'preproc__cat__handle_unknown', 'preproc__cat__max_categories', 'preproc__cat__min_frequency', 'preproc__cat__sparse_output', 'regression__alpha', 'regression__copy_X', 'regression__fit_intercept', 'regression__max_iter', 'regression__positive', 'regression__random_state', 'regression__solver', 'regression__tol'])

In [ ]:
from sklearn.model_selection import cross_validate

scoring = ['neg_mean_squared_error', 'neg_mean_absolute_error', 'r2', 'neg_root_mean_squared_error']

res_simple_dict = cross_validate(pipe_A, X_train, y_train, cv = 5, scoring = scoring)
res_simple = pd.DataFrame(res_simple_dict).assign(experiment = 1)
res_simple
# the scores for neg_mean_squared_error and neg_root_mean_squared_error don't seem very good, but we'll make do.

c:\Users\a6vij\Documents\DSI_Assignments\production\production-env\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\a6vij\Documents\DSI_Assignments\production\production-env\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


,fit_time,score_time,test_neg_mean_squared_error,test_neg_mean_absolute_error,test_r2,test_neg_root_mean_squared_error,experiment
0,0.014523,0.004511,-1668.454225,-17.726045,-0.022768,-40.846716,1
1,0.005999,0.003073,-370.380723,-15.390723,-0.712097,-19.245278,1
2,0.005001,0.004008,-946.166549,-17.540000,-0.313524,-30.759820,1
3,0.005001,0.005510,-7267.197630,-24.138251,-0.008662,-85.247860,1
4,0.005508,0.004001,-703.721398,-15.448546,-0.198628,-26.527748,1


# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [ ]:
param_grid_ridge_1 = {
    'regression__alpha': [1, 10, 100, 290, 500],
    'regression__fit_intercept': [True, False],
    'regression__solver': ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
}

param_grid_ridge_2 = {
    'regression__alpha': [0.01, 0.1, 1, 5, 10, 50, 100],
    'regression__fit_intercept': [True],
    'regression__solver': ['auto']
}

param_grid_ridge_3 = {
    'regression__alpha': [10, 100],
    'regression__fit_intercept': [True],
    'regression__solver': ['svd', 'cholesky', 'lsqr']
}

param_grid_ridge_4 = {
    'regression__alpha': [1, 10, 100],
    'regression__fit_intercept': [True, False],
    'regression__positive': [True]
}

param_grids_ridge = [ # create a list of parameter grids for the ridge experiments
    param_grid_ridge_1,
    param_grid_ridge_2,
    param_grid_ridge_3,
    param_grid_ridge_4
]

param_grid_ts_1 = {
    'regression__n_subsamples': [None, 50, 100],
    'regression__max_iter': [300, 500],
    'regression__tol': [1e-3, 1e-4]
}

param_grid_ts_2 = {
    'regression__n_subsamples': [None],
    'regression__max_iter': [500, 1000],
    'regression__tol': [1e-4, 1e-5]
}

param_grid_ts_3 = {
    'regression__n_subsamples': [50, 100, None],
    'regression__max_iter': [300, 500],
    'regression__tol': [1e-3]
}

param_grid_ts_4 = {
    'regression__n_subsamples': [None, 50],
    'regression__max_iter': [300, 500],
    'regression__tol': [1e-3]
}

param_grids_ts = [ # create a list of parameter grids for the TheilSenRegressor experiments
    param_grid_ts_1,
    param_grid_ts_2,
    param_grid_ts_3,
    param_grid_ts_4
]

def run_experiments(pipe, grids, X, y, cv=5): # function to run multiple gridsearch experiments, then print best params
    out = []
    for i, grid in enumerate(grids, start=1):
        gs = GridSearchCV(
            pipe,
            grid,
            cv=cv,
            scoring='neg_mean_squared_error',
            n_jobs=-1
        )
        gs.fit(X, y)

        df = pd.DataFrame(gs.cv_results_)
        df['experiment'] = i
        out.append(df)

        print(f"Experiment {i} best params:", gs.best_params_)
        
    return pd.concat(out, ignore_index=True)



In [14]:
run_experiments(
    pipe_A, param_grids_ridge, X_train, y_train, cv=5).sort_values(
        'mean_test_score', ascending=False).reset_index(drop=True)

Experiment 1 best params: {'regression__alpha': 500, 'regression__fit_intercept': True, 'regression__solver': 'lsqr'}
Experiment 2 best params: {'regression__alpha': 100, 'regression__fit_intercept': True, 'regression__solver': 'auto'}
Experiment 3 best params: {'regression__alpha': 100, 'regression__fit_intercept': True, 'regression__solver': 'svd'}
Experiment 4 best params: {'regression__alpha': 100, 'regression__fit_intercept': True, 'regression__positive': True}


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regression__alpha,param_regression__fit_intercept,param_regression__solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,experiment,param_regression__positive
0,0.009685,0.001752,0.003910,0.000373,100.00,True,NaN,"{'regression__alpha': 100, 'regression__fit_in...",-1598.260797,-268.929853,-761.846195,-7172.247326,-589.069359,-2078.070706,2584.701746,1,4,True
1,0.012447,0.006439,0.003505,0.000632,500.00,True,lsqr,"{'regression__alpha': 500, 'regression__fit_in...",-1607.891275,-254.888342,-737.649967,-7225.016789,-592.373644,-2083.564003,2609.214029,1,1,NaN
2,0.014845,0.003634,0.005216,0.000757,500.00,True,sag,"{'regression__alpha': 500, 'regression__fit_in...",-1607.901382,-254.907006,-737.650613,-7225.011464,-592.365095,-2083.567112,2609.209856,2,1,NaN
3,0.013838,0.002325,0.004831,0.000529,500.00,True,saga,"{'regression__alpha': 500, 'regression__fit_in...",-1607.900825,-254.905117,-737.651130,-7225.016588,-592.363944,-2083.567521,2609.212239,3,1,NaN
4,0.012733,0.003150,0.005318,0.001747,500.00,True,svd,"{'regression__alpha': 500, 'regression__fit_in...",-1607.900653,-254.906640,-737.649713,-7225.016164,-592.365155,-2083.567665,2609.211872,4,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,0.018503,0.003753,0.006686,0.003463,1.00,True,sparse_cg,"{'regression__alpha': 1, 'regression__fit_inte...",-1668.489424,-370.376381,-946.298159,-7267.205798,-703.736158,-2191.221184,2573.588512,58,1,NaN
75,0.021109,0.005765,0.005721,0.000817,1.00,True,lsqr,"{'regression__alpha': 1, 'regression__fit_inte...",-1668.696900,-370.834085,-950.523973,-7267.464347,-704.173121,-2192.338485,2573.158399,59,1,NaN
76,0.016339,0.005752,0.006115,0.000490,1.00,False,lsqr,"{'regression__alpha': 1, 'regression__fit_inte...",-1668.454413,-370.743367,-952.431145,-7266.989200,-703.205707,-2192.364766,2572.921679,60,1,NaN
77,0.009025,0.001522,0.003935,0.000579,0.10,True,auto,"{'regression__alpha': 0.1, 'regression__fit_in...",-1663.781213,-380.962657,-1085.423561,-7269.736446,-719.639678,-2223.908711,2558.497382,6,2,NaN


In [16]:
run_experiments(
    pipe_B, param_grids_ridge, X_train, y_train, cv=5).sort_values(
        'mean_test_score', ascending=False).reset_index(drop=True)

Experiment 1 best params: {'regression__alpha': 500, 'regression__fit_intercept': True, 'regression__solver': 'sag'}
Experiment 2 best params: {'regression__alpha': 100, 'regression__fit_intercept': True, 'regression__solver': 'auto'}
Experiment 3 best params: {'regression__alpha': 100, 'regression__fit_intercept': True, 'regression__solver': 'svd'}
Experiment 4 best params: {'regression__alpha': 100, 'regression__fit_intercept': True, 'regression__positive': True}


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regression__alpha,param_regression__fit_intercept,param_regression__solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,experiment,param_regression__positive
0,0.037374,0.002777,0.005472,0.000640,100.00,True,NaN,"{'regression__alpha': 100, 'regression__fit_in...",-1595.091710,-274.350803,-763.716884,-7177.084995,-586.667922,-2079.382463,2586.055705,1,4,True
1,0.052188,0.006160,0.006576,0.001086,500.00,True,sag,"{'regression__alpha': 500, 'regression__fit_in...",-1610.829183,-258.605038,-741.310090,-7233.281725,-590.726093,-2086.950426,2611.654621,1,1,NaN
2,0.047761,0.011814,0.012609,0.006810,500.00,True,lsqr,"{'regression__alpha': 500, 'regression__fit_in...",-1610.829823,-258.604369,-741.312685,-7233.278419,-590.730465,-2086.951152,2611.652620,2,1,NaN
3,0.046300,0.004045,0.007125,0.002097,500.00,True,saga,"{'regression__alpha': 500, 'regression__fit_in...",-1610.829633,-258.604939,-741.312177,-7233.281723,-590.727370,-2086.951168,2611.654257,3,1,NaN
4,0.043683,0.006963,0.006744,0.000536,500.00,True,svd,"{'regression__alpha': 500, 'regression__fit_in...",-1610.829273,-258.605358,-741.312088,-7233.287364,-590.727307,-2086.952278,2611.656451,4,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,0.037223,0.006717,0.006912,0.000981,1.00,True,cholesky,"{'regression__alpha': 1, 'regression__fit_inte...",-1651.423186,-371.073941,-839.044457,-7241.870741,-699.838393,-2160.650144,2575.317860,58,1,NaN
75,0.034726,0.005575,0.005965,0.000912,1.00,True,auto,"{'regression__alpha': 1, 'regression__fit_inte...",-1651.423186,-371.073941,-839.044457,-7241.870741,-699.838393,-2160.650144,2575.317860,5,2,NaN
76,0.039270,0.008474,0.009481,0.002980,1.00,True,lsqr,"{'regression__alpha': 1, 'regression__fit_inte...",-1651.025381,-371.175253,-839.411493,-7242.496717,-699.912157,-2160.804200,2575.520503,60,1,NaN
77,0.028800,0.004628,0.006594,0.002630,0.10,True,auto,"{'regression__alpha': 0.1, 'regression__fit_in...",-1646.331463,-386.778646,-853.247918,-7240.740484,-716.993097,-2168.818322,2569.497006,6,2,NaN


In [17]:
run_experiments(
    pipe_C, param_grids_ts, X_train, y_train, cv=5).sort_values(
        'mean_test_score', ascending=False).reset_index(drop=True)

Experiment 1 best params: {'regression__max_iter': 300, 'regression__n_subsamples': None, 'regression__tol': 0.0001}
Experiment 2 best params: {'regression__max_iter': 1000, 'regression__n_subsamples': None, 'regression__tol': 1e-05}
Experiment 3 best params: {'regression__max_iter': 500, 'regression__n_subsamples': None, 'regression__tol': 0.001}
Experiment 4 best params: {'regression__max_iter': 500, 'regression__n_subsamples': None, 'regression__tol': 0.001}


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regression__max_iter,param_regression__n_subsamples,param_regression__tol,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,experiment
0,2.004406,0.057264,0.005233,0.000512,300,None,0.00010,"{'regression__max_iter': 300, 'regression__n_s...",-1651.858615,-245.313243,-750.301727,-7316.334335,-607.960888,-2114.353762,2641.880904,1,1
1,1.982591,0.113163,0.006451,0.001998,500,None,0.00100,"{'regression__max_iter': 500, 'regression__n_s...",-1649.459713,-242.777877,-748.380502,-7322.417191,-610.338490,-2114.674755,2644.647115,1,4
2,2.310552,0.164697,0.006268,0.001672,1000,None,0.00001,"{'regression__max_iter': 1000, 'regression__n_...",-1648.220689,-244.113842,-752.075993,-7323.529708,-607.344244,-2115.056895,2644.899520,1,2
3,1.957493,0.196769,0.006122,0.000868,300,None,0.00100,"{'regression__max_iter': 300, 'regression__n_s...",-1650.191256,-245.762306,-750.249109,-7322.437428,-607.094976,-2115.147015,2644.383623,2,4
4,2.114037,0.100170,0.007931,0.001165,500,None,0.00001,"{'regression__max_iter': 500, 'regression__n_s...",-1649.080631,-244.032622,-754.163421,-7322.259199,-606.274049,-2115.161985,2644.287360,2,2
5,2.313724,0.099060,0.006331,0.000926,500,None,0.00010,"{'regression__max_iter': 500, 'regression__n_s...",-1651.392368,-246.165371,-753.383372,-7320.441686,-605.739634,-2115.424486,2643.329864,3,2
6,1.784544,0.076870,0.004861,0.000529,500,None,0.00100,"{'regression__max_iter': 500, 'regression__n_s...",-1651.422448,-244.119087,-749.550647,-7324.404544,-608.921581,-2115.683661,2645.211411,2,1
7,2.002314,0.175740,0.005010,0.001047,300,None,0.00100,"{'regression__max_iter': 300, 'regression__n_s...",-1647.082020,-242.952442,-753.438044,-7324.785080,-610.555414,-2115.762600,2645.092491,3,1
8,2.044156,0.136857,0.005962,0.002225,500,None,0.00010,"{'regression__max_iter': 500, 'regression__n_s...",-1649.857349,-244.601277,-754.189347,-7324.334232,-606.521311,-2115.900703,2644.965850,4,1
9,1.482843,0.069253,0.005851,0.001326,500,None,0.00100,"{'regression__max_iter': 500, 'regression__n_s...",-1651.386165,-244.925384,-750.039027,-7324.214371,-609.051180,-2115.923226,2644.958497,1,3


In [18]:
run_experiments(
    pipe_D, param_grids_ts, X_train, y_train, cv=5).sort_values(
        'mean_test_score', ascending=False).reset_index(drop=True)

Experiment 1 best params: {'regression__max_iter': 500, 'regression__n_subsamples': 50, 'regression__tol': 0.0001}
Experiment 2 best params: {'regression__max_iter': 1000, 'regression__n_subsamples': None, 'regression__tol': 1e-05}
Experiment 3 best params: {'regression__max_iter': 300, 'regression__n_subsamples': 50, 'regression__tol': 0.001}
Experiment 4 best params: {'regression__max_iter': 300, 'regression__n_subsamples': None, 'regression__tol': 0.001}


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regression__max_iter,param_regression__n_subsamples,param_regression__tol,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,experiment
0,2.416720,0.279609,0.007949,0.001960,1000,None,0.00001,"{'regression__max_iter': 1000, 'regression__n_...",-1641.576221,-240.862068,-738.894594,-7315.692246,-592.117779,-2105.828582,2645.605363,1,2
1,2.778287,0.282205,0.008317,0.001509,300,50,0.00100,"{'regression__max_iter': 300, 'regression__n_s...",-1628.141241,-270.363632,-740.912557,-7284.183331,-611.442681,-2107.008689,2627.114489,1,3
2,2.465455,0.251409,0.007125,0.000960,300,None,0.00100,"{'regression__max_iter': 300, 'regression__n_s...",-1640.935789,-242.381430,-736.941840,-7323.961022,-592.409069,-2107.325830,2648.839183,2,3
3,2.343531,0.181759,0.006831,0.001211,300,None,0.00100,"{'regression__max_iter': 300, 'regression__n_s...",-1645.008805,-240.478724,-737.173044,-7320.140420,-594.125083,-2107.385215,2647.239552,1,4
4,2.410672,0.307787,0.009220,0.002471,500,None,0.00001,"{'regression__max_iter': 500, 'regression__n_s...",-1644.010392,-240.896472,-735.464652,-7320.233009,-596.571403,-2107.435186,2647.149522,2,2
5,2.397902,0.251378,0.009417,0.004292,500,None,0.00010,"{'regression__max_iter': 500, 'regression__n_s...",-1640.788732,-239.599671,-738.157938,-7323.553610,-595.587363,-2107.537463,2648.586937,3,2
6,3.063180,0.231399,0.007120,0.000379,500,50,0.00010,"{'regression__max_iter': 500, 'regression__n_s...",-1626.225376,-271.040672,-744.995805,-7286.319377,-609.747557,-2107.665758,2627.700827,1,1
7,3.100160,0.184503,0.009735,0.003504,300,50,0.00010,"{'regression__max_iter': 300, 'regression__n_s...",-1630.939647,-269.009577,-745.793129,-7284.440420,-609.143130,-2107.865181,2627.058704,2,1
8,2.048032,0.084970,0.006624,0.000673,500,None,0.00100,"{'regression__max_iter': 500, 'regression__n_s...",-1640.533025,-244.646787,-738.285107,-7320.468088,-595.589639,-2107.904529,2646.655931,3,3
9,2.942222,0.147785,0.009238,0.001924,500,50,0.00100,"{'regression__max_iter': 500, 'regression__n_s...",-1628.516207,-269.661080,-743.872000,-7286.211078,-612.276193,-2108.107312,2627.595841,4,3


# Evaluate

+ Which model has the best performance?

My best performance was Experiment 4 best params: {'regression__alpha': 100, 'regression__fit_intercept': True, 'regression__positive': True}, with a mean test score of -2078.070706. This still isn't great, as I'd expected Theil-Sen to be more robust against the outliers in the dataset.

# Export

+ Save the best performing model to a pickle file.

In [19]:
with open("best_model.pkl", "wb") as f:
    pickle.dump(pipe_A, f)

# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

c:\Users\a6vij\Documents\DSI_Assignments\production\production-env\Scripts\python.exe


In [ ]:
import shap

# Unfortunately, no matter what I try, I am unable to get the venv to install the shap package.
# I am unable to run the SHAP analysis.

ModuleNotFoundError: No module named 'shap'

*(Answer here.)*

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at the `help` channel. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.